In [133]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [138]:
class SimpleNN(nn.Module):
    def __init__(self
                 , vocab_size
                 , sequence_length
                 , hidden_dim
                 , word_embed
                 , interaction_in
                 , interaction_out
                 ,sentence_embed
                ):
        super(SimpleNN, self).__init__()
        self.vocab_size = vocab_size
        self.sequence_length = sequence_length
        self.word_embed = word_embed
        self.interaction_in = interaction_in
        
        self.lin1 = nn.Linear(vocab_size,word_embed) #50 is word embeddings essentially
        self.lin2 = nn.Linear(word_embed**2, interaction_in)
        self.lin3 = nn.Linear(interaction_in**2,interaction_out)
        self.lin4 = nn.Linear(sequence_length,sentence_embed)
        self.lin5 = nn.Linear(interaction_out*sentence_embed,vocab_size)
        # Activation function

    def forward(self, x):
        print(x.shape)
        x = F.relu(self.lin1(x))
        print(x.shape)
        # Flatten the sequence_length and vocab_size dimensions
        x = F.relu(torch.einsum('bij,bik->bijk', x, x)) #interactions
        print(x.shape)
        x = x.view(x.size(0), self.sequence_length, -1)
        print(x.shape)
        x = F.relu(self.lin2(x))
        print(x.shape)
        x = F.relu(torch.einsum('bij,bik->bijk', x, x)) #interactions x interactions 
        print(x.shape)
        x = x.view(x.size(0), self.sequence_length, -1)
        print(x.shape)
        x = F.relu(self.lin3(x))
        print(x.shape)
        x = x.permute(0, 2, 1)
        print(x.shape)
        x = F.relu(self.lin4(x))
        print(x.shape)
        x = x.view(x.size(0), -1)
        print(x.shape)
        x = self.lin5(x)
        print(x.shape)
        # Pass through the second fully connected layer
        x = F.softmax(x, dim=1)

        return x

In [139]:
# Hyperparameters
vocab_size = 212  # Example vocab size
sequence_length = 347  # Example sequence length
hidden_dim = 128  # Example hidden dimension
word_embed = 51
interaction_in = 52
interaction_out = 53
sentence_embed = 54
batch_size = 50

# Instantiate the model
model = SimpleNN(vocab_size, sequence_length, hidden_dim, word_embed, interaction_in,interaction_out,sentence_embed)
print(model)

SimpleNN(
  (lin1): Linear(in_features=212, out_features=51, bias=True)
  (lin2): Linear(in_features=2601, out_features=52, bias=True)
  (lin3): Linear(in_features=2704, out_features=53, bias=True)
  (lin4): Linear(in_features=347, out_features=54, bias=True)
  (lin5): Linear(in_features=2862, out_features=212, bias=True)
)


In [140]:
rand_input = torch.rand(batch_size, sequence_length, vocab_size)
rand_input.shape

torch.Size([50, 347, 212])

In [142]:
model(rand_input).shape

torch.Size([50, 347, 212])
torch.Size([50, 347, 51])
torch.Size([50, 347, 51, 51])
torch.Size([50, 347, 2601])
torch.Size([50, 347, 52])
torch.Size([50, 347, 52, 52])
torch.Size([50, 347, 2704])
torch.Size([50, 347, 53])
torch.Size([50, 53, 347])
torch.Size([50, 53, 54])
torch.Size([50, 2862])
torch.Size([50, 212])


torch.Size([50, 212])

In [143]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Using the SimpleNN model define

In [144]:
total_params = count_parameters(model)
print(f"Total number of trainable parameters: {total_params}")

Total number of trainable parameters: 915280
